# driver
driver.page_source  
driver.current_url  
driver.find_element_by_id("id").click()
# beautifulsoup
a['href']  
soup.find_all('b', class_="pos")  
a.text 可為該tag中的text，若是tag中間含有其他tag，他們的text也會被包含在內  
lear_examp.div.extract() 在lear_examp中將div抽出來並返回  
lear_examp.div.decompose() 在lear_examp中將div抽出來摧毀並不返回
# json
json.dumps([item.reprJSON() for item in word_list], cls=ComplexEncoder)  
``` python
class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj,'reprJSON'):
            return obj.reprJSON()
        else:
            return json.JSONEncoder.default(self, obj)
```
# re
利用re模块分割含有多种分割符的字符串：
``` python
import re
a='Beautiful, is; better*than\nugly'
x= re.split(',|; |\*|\n',a)
print(x)
```

In [32]:
%autosave 180

Autosaving every 180 seconds


In [33]:
from selenium import webdriver
import time

In [60]:
username = "englishprofile"
password = "vocabulary"
url = "http://" + username + ":" + password + "@vocabulary.englishprofile.org/"

# PhantomJS

In [4]:
# driver = webdriver.PhantomJS(executable_path=r'/usr/local/share/phantomjs-2.1.1-linux-x86_64/bin/phantomjs')  # PhantomJs
# driver.get(url)  # 輸入範例網址，交給瀏覽器 
# pageSource = driver.page_source  # 取得網頁原始碼
# print(pageSource)

# driver.close() 

# Chrome

In [68]:
driver = webdriver.Chrome(executable_path=r'/usr/local/share/chrome_driver/chromedriver')
#driver = webdriver.Chrome(executable_path=r'C:\Users\USER\Desktop\chromedriver')
driver.get(url)
time.sleep(2)

# from uk to us
driver.find_element_by_id("lang-us-btn").click()
time.sleep(2)

# browse a-z
driver.find_element_by_id("id_browse").click()
time.sleep(2)
a2z_html = driver.page_source  # 取得網頁原始碼

In [69]:
from bs4 import BeautifulSoup
import re
import json

alpha_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',
         'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y' ,'Z']
prefix_alpha = 'http://vocabulary.englishprofile.org/dictionary//word-list/us/a1_c2/'
prefix_sub = 'http://vocabulary.englishprofile.org'

# unused
pos_dict = {'determiner':'-det.', 'verb':'-v.', 'adjective':'-adj.', 'adverb':'-adv.', 
            'noun': '-n.', 'exclamation': '-excl.', 'conjunction': '-conj.', 
            'pronoun': '-pron.', 'preposition': '-prep.', }

a2z: show a to z link  
list: means some prefix alpha, which seperate several word into n sublists  
sublist: composed by several words  
span_list: list every words


## json 格式  
- class Word  
    - str word  
    - POS list poses  
- class POS   
    - str pos  
    - Sense list senses  
- class Sense  
    - str sense  
    - str level  
    - str dict_examp  
    - str lear_examp

In [37]:
class Word(object):
    def __init__(self, word, poses):
        self.word = word
        self.poses = poses
    def reprJSON(self):
        return dict(word=self.word, poses=self.poses)
    
class POS(object):
    def __init__(self, pos, senses):
        self.pos = pos
        self.senses = senses
    def reprJSON(self):
        return dict(pos=self.pos, senses=self.senses) 

class Sense(object):
    def __init__(self, sense, level, dict_examp, lear_examp):
        self.sense = sense
        self.level = level
        self.dict_examp = dict_examp
        self.lear_examp = lear_examp
    def reprJSON(self):
        return dict(sense=self.sense, level=self.level, dict_examp=self.dict_examp, lear_examp=self.lear_examp) 

class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj,'reprJSON'):
            return obj.reprJSON()
        else:
            return json.JSONEncoder.default(self, obj)

## html 內容
- a class = 'lookup' : 單字
- b class = 'pos': 詞性
    - div class = 'sense'
        - span class = 'def': 由多個 a 組成
        - span class = 'freq-A1': 分級
        - blockqoute class = 'examp': dict example 有的沒有
        - blockqoute class = 'clc': learner example 有的沒有
        
popcorn no pos

In [38]:
# remove [ ] \n \t
def remove_char(string):
    string = re.sub('\[|\]|\n|\t', '', string)
    return string

In [9]:
for index, alpha in enumerate(alpha_list):
    print('processing %s' % alpha)
    if alpha not in 'IPS': continue #miss i
    alpha_url = prefix_alpha + alpha
    driver.get(alpha_url)
    time.sleep(2)
    alpha_html = driver.page_source
    
    soup_a2z = BeautifulSoup(alpha_html, 'html.parser')
    alpha_list = soup_a2z.find_all('a')
    
    # remove some unneeded url
    temp_list = []
    for sublist in alpha_list:
        if (re.match('/dictionary/word-list/us/a1_c2/' + alpha + '/', sublist['href'])):
            temp_list.append(sublist)
    alpha_list = temp_list
    
    word_vocalbulary_list = []
    
    for sublist in alpha_list:
        # crawl every sublist
        sublist_url = prefix_sub + sublist['href']
        driver.get(sublist_url)
        time.sleep(2)
        sublist_html = driver.page_source

        soup_sublist = BeautifulSoup(sublist_html, 'html.parser')
        span_list = soup_sublist.find_all('a')

        # remove some unneeded url
        temp_list = []
        for word in span_list:
            if(word['href'][-1] == '#' and word['href'] != '#'):
                temp_list.append(word)
        span_list = temp_list
        
        for word in span_list:
            word_url = prefix_sub + word['href']
            driver.get(word_url)
            time.sleep(2)
            word_html = driver.page_source
            
            soup_word = BeautifulSoup(word_html, 'html.parser')
           
            # lookup word 
            lookup = soup_word.find('h1', class_='hw').text
            
            word_pos_list= []
            
            # posblocks
            posblocks = soup_word.find_all('div', class_='posblock')
            for posblock in posblocks:
                # pos
                pos = posblock.find('b', class_='pos')
                if(pos != None):
                    pos = pos.text
                else:
                    print(lookup + " has no pos.")
                    pos = ""
                
                word_sense_list = []
                
                # gwblocks
                gwblocks = posblock.find_all('div', class_='gwblock')
                for gwblock in gwblocks:
                    # sense
                    sense = gwblock.find('span', class_='def')
                    sense_str = sense.text.strip()
                    
                    # level
                    level = gwblock.find('span', class_=re.compile('freq-'))
                    level = level.text
                    
                    # dict examp
                    dict_examp = gwblock.find('blockquote', class_='examp')
                    if(dict_examp != None):
                        dict_examp_str = remove_char(dict_examp.text.strip())
                    else:
                        dict_examp_str = ""
                        
                    # learner examp
                    lear_examp = gwblock.find('blockquote', class_='clc')
                    if(lear_examp != None):
                        lear_examp_str = lear_examp.text.strip()
                        lear_examp.div.decompose() # remove "Learner example:"
                        try: # some learn'examp directly comes up with examp
                            lear_examp.div.decompose() # remove examp come from    
                        except:
                            pass
                        lear_examp_str = remove_char(lear_examp.text.strip())
                    else:
                        lear_examp_str = ""
                        
                    word_sense = Sense(sense_str, level, dict_examp_str, lear_examp_str)
                    word_sense_list.append(word_sense)
                word_pos = POS(pos, word_sense_list)
                word_pos_list.append(word_pos)
            word_vocalbulary = Word(lookup, word_pos_list)
            word_vocalbulary_list.append(word_vocalbulary)
            
    with open( 'data/' + alpha + '.json', 'w') as outfile:  
        outfile.write(json.dumps([item.reprJSON() for item in word_vocalbulary_list], cls=ComplexEncoder))

processing A
processing B
processing C
processing D
processing E
processing F
processing G
processing H
processing I


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
driver.close() 

# temp test


In [70]:
word_lists = []

for index, alpha in enumerate(alpha_list):
    print('processing %s' % alpha)
    if alpha != 'S': continue #miss i
    alpha_url = prefix_alpha + alpha
    driver.get(alpha_url)
    time.sleep(2)
    alpha_html = driver.page_source
    
    soup_a2z = BeautifulSoup(alpha_html, 'html.parser')
    alpha_list = soup_a2z.find_all('a')
    
    # remove some unneeded url
    temp_list = []
    for sublist in alpha_list:
        if (re.match('/dictionary/word-list/us/a1_c2/' + alpha + '/', sublist['href'])):
            temp_list.append(sublist)
    alpha_list = temp_list
    
    word_vocalbulary_list = []
    
    for sublist in alpha_list:
        # crawl every sublist
        sublist_url = prefix_sub + sublist['href']
        driver.get(sublist_url)
        time.sleep(2)
        sublist_html = driver.page_source

        soup_sublist = BeautifulSoup(sublist_html, 'html.parser')
        span_list = soup_sublist.find_all('a')

        # remove some unneeded url
        temp_list = []
        for word in span_list:
            if(word['href'][-1] == '#' and word['href'] != '#'):
                # get all word's href
                word_lists.append(prefix_sub+word['href'])
                temp_list.append(word)
        span_list = temp_list

processing A
processing B
processing C
processing D
processing E
processing F
processing G
processing H
processing I
processing J
processing K
processing L
processing M
processing N
processing O
processing P
processing Q
processing R
processing S
processing T
processing U
processing V
processing W
processing X
processing Y
processing Z


In [64]:
len(word_lists)

786

In [77]:

for i, word in enumerate(word_lists):
    if i < 580: continue
    word_url = word
    driver.get(word_url)
    time.sleep(2)
    word_html = driver.page_source

    soup_word = BeautifulSoup(word_html, 'html.parser')

    # lookup word 
    lookup = soup_word.find('h1', class_='hw').text

    word_pos_list= []

    # posblocks
    posblocks = soup_word.find_all('div', class_='posblock')
    for posblock in posblocks:
        # pos
        pos = posblock.find('b', class_='pos')
        if(pos != None):
            pos = pos.text
        else:
            print(lookup + " has no pos.")
            pos = ""

        word_sense_list = []

        # gwblocks
        gwblocks = posblock.find_all('div', class_='gwblock')
        for gwblock in gwblocks:
            # sense
            sense = gwblock.find('span', class_='def')
            sense_str = sense.text.strip()

            # level
            level = gwblock.find('span', class_=re.compile('freq-'))
            level = level.text

            # dict examp
            dict_examp = gwblock.find('blockquote', class_='examp')
            if(dict_examp != None):
                dict_examp_str = remove_char(dict_examp.text.strip())
            else:
                dict_examp_str = ""

            # learner examp
            lear_examp = gwblock.find('blockquote', class_='clc')
            if(lear_examp != None):
                lear_examp_str = lear_examp.text.strip()
                lear_examp.div.decompose() # remove "Learner example:"
                try: # some learn'examp directly comes up with examp
                    lear_examp.div.decompose() # remove examp come from    
                except:
                    pass
                lear_examp_str = remove_char(lear_examp.text.strip())
            else:
                lear_examp_str = ""

            word_sense = Sense(sense_str, level, dict_examp_str, lear_examp_str)
            word_sense_list.append(word_sense)
        word_pos = POS(pos, word_sense_list)
        word_pos_list.append(word_pos)
    word_vocalbulary = Word(lookup, word_pos_list)
    word_vocalbulary_list.append(word_vocalbulary)

with open( 'data/' + 'S' + '.json', 'w') as outfile:  
    outfile.write(json.dumps([item.reprJSON() for item in word_vocalbulary_list], cls=ComplexEncoder))

In [75]:
print(i)

580


In [76]:
print(len(word_vocalbulary_list))

580
